# Comparing MASH analysis with simple multivariate analysis

Previously we showed that even though univariate analysis with degenerated MASH model gives identical results to SuSiE as expected (with non-decreasing ELBO), in multivariate calculations the ELBO is not always non-decreasing. To investigate the issue we will 1) further simplify the problem and 2) isolate the problem to posterior calculation versus ELBO calculations and check which part is problematic. The best way to achieve both is to implement a simple Bayesian multivariate regression model with prior $b \sim MVN(0, U)$ where $U$ is known, instead of using MASH prior for $b$.

This feature is now implemented in the [`BayesianMultivariateRegression` class](https://github.com/gaow/mmbr/blob/master/R/bayesian_multivariate_regression.R) with an interface added to the main function such that the code will be triggered [when prior variance is a matrix](https://github.com/gaow/mmbr/blob/dccc26cfbba0cfd3e6f20b37e06d4db6305b834f/R/main.R#L110).

## Test the calculation agree with univariate code

In [1]:
library(mmbr)
set.seed(2)
L = 5

Loading required package: mashr
Loading required package: ashr


In [2]:
dat = mmbr_sim1(r=1)

Run the simulated univariate data with SuSiE,

In [3]:
res = susieR::susie(dat$X,dat$y,L=L,scaled_prior_variance=0.2,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

Now run it with multivariate simple prior implementation in `mmbr`,

In [4]:
res = mmbr::susie(dat$X,dat$y,L=L,V=0.2*var(dat$y),compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

So it is confirmed that this implementation produces identical results as SuSiE runs.

## Test multivariate analysis

In [5]:
set.seed(2)
dat = mmbr_sim1(r=5)

In [7]:
res = mmbr::susie(dat$X,dat$y,L=L,V=dat$V,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2184.076 -2173.199 -2169.551 -2167.744 -2167.267 -2167.087 -2166.514
 [8] -2166.013 -2164.894 -2163.635 -2163.574 -2163.573 -2163.573

Here the ELBO is non-decreasing, as expected.

## Compare with MASH based model

Here to keep things simple we use simulation with 2 conditions, and a diagonal prior covariance. We analyze with simple Bayesian multivariate regression,

In [8]:
set.seed(2)
dat = mmbr_sim1(r=2)
res = mmbr::susie(dat$X,dat$y,L=L,V=0.2*diag(2),compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -857.2792 -843.5537 -842.7213 -842.3799 -842.3469 -842.3417 -842.3405
[8] -842.3402

and with MASH based regression,

In [9]:
m_init = mmbr:::MashInitializer$new(list(0.2*diag(2)), 1, prior_weight=1, null_weight=0,alpha=0)

In [10]:
res = mmbr::susie(dat$X,dat$y,L=L,V=m_init,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

Warning message in m$get_objective(dump = TRUE):
“Objective is not non-decreasing”

[1] -857.9446 -843.1662 -842.3576 -842.3300 -842.3242 -842.3243

So for now, the results do not agree with MASH. But with this mininal example and baseline comparison we should be in a much better position to dig into what's going on.